In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import numpy as np
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

class PrcCargaInterfaces():
  
  def __init__(self, id_solicitud):
    self.id_solicitud = id_solicitud
    
    #Obteniendo parametros
    parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
    self.fecha_proceso = parametros['fecha_proceso']  
    self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d')
    self.archivo_entrada = parametros['archivo_entrada']
    self.tabla_destino = parametros['tabla_destino']
    self.tabla_destino_2 = parametros['tabla_destino_2']
    self.ruta_origen = '/dbfs/mnt/ereportinterfaz/inseir/'
        
    #Fecha de actualizacion
    fecha_actualizacion = datetime.now()
    self.fecha_actualizacion = fecha_actualizacion
    self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
    
    #Conexion
    connection = PrcGeneral().databaseConnect()
    self.connection = connection
    
    cursor = self.connection.cursor()
    self.cursor = cursor
    
  def cargarInterfaceCastSalVencidos(self):

      spark = SparkSession.builder.appName("Cargar interface CAST_SAL_VENCIDOS").getOrCreate()
      df = spark.read.text('/mnt/ereportinterfaz/inseir/' + self.archivo_entrada)
      df = df.select(
        df.value.substr(1,8).alias('FECHA'),    
        df.value.substr(9,12).alias('CTA_CMR'),    
        df.value.substr(21,3).alias('COD_PRODUCTO'),    
        df.value.substr(24,9).alias('COD_CLIENTE'),  
        df.value.substr(33,1).alias('TIPO_DOC'),  
        df.value.substr(34,12).alias('NUM_DOC'),  
        df.value.substr(46,120).alias('NOMBRE'),  
        df.value.substr(166,20).alias('COD_OPERACION'),  
        df.value.substr(186,15).alias('SALDO'),  
        df.value.substr(201,4).alias('DIAS_MORA'),  
        df.value.substr(204,22).alias('IMPORTE'),  
        df.value.substr(227,15).alias('DEUDA_TOTAL'),  
        df.value.substr(242,8).alias('FEC_CASTIGO'))
      url = "jdbc:sqlserver://auqui.database.windows.net;database=dbnormativo"
      usuario = "administrador"
      clave = "Auqui$2020"
      try:
        df.write.format("com.microsoft.sqlserver.jdbc.spark").mode("overwrite").option("truncate", "true").option("url", url).option("dbtable", self.tabla_destino).option("user", usuario).option("password", clave).option("tableLock", "true").save()
      except:
          raise;
        
PrcCargaInterfaces = PrcCargaInterfaces(20)
PrcCargaInterfaces.cargarInterfaceCastSalVencidos()